# Optimization Sample: Shipping

This notebook follows a sample shipping company that has a difficult business problem: balancing the loads of container ships at port. We'll apply the power of QIO to generate a solution to a simplified version of this problem.

## Pre-requisites

1. [Create an Azure Quantum Workspace](https://github.com/MicrosoftDocs/quantum-docs-private/wiki/Create-quantum-workspaces-with-the-Azure-portal)
2. [Install the `azure-quantum` Python module](https://github.com/MicrosoftDocs/quantum-docs-private/wiki/Use-the-Python-SDK-for-Quantum-Inspired-Optimization).


## The Problem
Contoso Logistics Inc is a world class freight company operating a fleet of large container ships. When these ships are docked at port Contoso Logistics needs to distribute containers between the two ships as evenly as possible. If they make one ship significantly heavier than the other then it will move slower and consume more fuel - delaying shipments and costing a significant amount of money! To make things harder, the weights of individual containers can vary greatly so it's not easy to divide them.

This problem is known as the Number partitioning problem and is NP-complete. It is however relatively straightforward to apply Quantum Inspired Optimization (QIO) to it to generate a good solution. Let's explore how.

## What does it mean to Optimize?
We're using Quantum-Inspired Optimization to solve this problem - but what does that actually mean?

In our case, optimize means "to find a solution with minimal cost" - which is exactly what the Solvers in Azure Quantum do. They take a representation of a difficult problem and apply techniques from physics to find a solution with the least cost.

> Note that solvers can be applied to very difficult problems where it's infeasible to confirm whether the solution found is the best solution that exists. So keep in mind that there may be better solutions than those returned by the solvers.

The solvers in Azure Quantum expect a `Binary Optimization Problem`, which is a problem expressed in the following (simplified) format:
$$  H = \Large\sum_{i} w_{i}x_{i} + \sum_{i,j} w_{i,j}x_{i}x_{j} $$
$$
x_{i} \in \left\{
        \begin{array}{ll}
            0 & \quad \\
            1 & \quad
        \end{array}
    \right.
\text{or}\quad
x_{i} \in \left\{
        \begin{array}{ll}
            1 & \quad \\
            -1 & \quad
        \end{array}
    \right.
$$

That is, a summation that is composed of weights ${w}$, and binary variables ${x_i}$. Let's explore how you formulate such a problem.


## Understanding the Problem
In the number partitioning problem, we have a set ${W}$ of container weights which we would like to partition into two sets: ${W_a}$ and ${W_b}$ (containers on the ships, ${a}$ and ${b}$). In this section, our goal will be to develop a representation of the problem that we can provide to the QIO solver.

## Breaking down the Problem
Let's start by coming up with an equation for the weight of a given ship, which is the sum of all the containers on the ship. This is expressed in the below equation, where ${w_i}$ is the weight of container ${i}$:

$$ \Large\sum_{i} w_{i} $$

Ideally, we'd like a solution where the weight difference between the ships is as small as possible. This is expressed by the following expression:

$$ H = \Large\sum_{i \in A} w_i - \Large\sum_{i \in B} w_i $$

If the value of ${H}$ is zero, we know the ships are equally loaded.

Next, we'll introduce a variable, ${x_i}$, to represent whether an individual container ${i}$ is assigned to ship ${a}$ or ship ${b}$. Because we can assign the container ${i}$ to either ship, the variable ${x_i}$ can be take on two different values - which makes it a `binary` variable. For convenience, we'll say the two values it can take on are ${1}$ and ${-1}$. ${1}$ will represent that the container is placed on ship ${a}$, and ${-1}$ will represent that the container is placed on ship ${b}$.

> Because of our choice to make ${i}$ be either ${1}$ or ${-1}$ this type of problem is called an `Ising` problem.

By introducing this variable ${x_i}$ to the previous equation, it can be simplified to:

$$ H = \Large\sum_{i \in A \cup B} w_ix_i $$

where $$ x_i = \begin{cases} +1 & \textrm{if   }i \in A \\ -1 & \textrm{if   }i \in B \end{cases} $$

The function ${H}$ will be called our `cost function` as it describes the cost of a given solution.

> The letter ${H}$ is traditionally used to represent a cost function and is also referred to as a `Hamiltonian` in a nod towards the quantum mechanical roots of Quantum-Inspired Optimization techniques.

## The final model
There's one last change we need to make before we can solve our problem. If we look at our cost function ${H}$ there's a flaw: the solution with the _least_ cost is to simply assign all containers to ship ${b}$ by setting all ${x_i}={-1}$ - that's not right! To fix this we'll take a simple step - we'll square the right hand side of the equation so that it cannot be negative:

$$ H^{2} = \Large(\sum_{i \in A \cup B} w_{i} x_{i})^{2} $$

This is somewhat arbitrary, but it yields a cost function with the right properties:
- If all the containers are on one ship, the function is at its highest value - reflecting that this is the least optimal solution
- If the containers are perfectly balanced, the value of the summation inside the square is ${0}$ - the function is at its lowest value
- In this case, we don't care about the actual value of ${H}$, just that it's as small as possible.

Next, let's express this problem in Python and solve it for a few cases.


## Solving the Problem in Python

First, we must instantiate a `Workspace` object which allows you to connect to the Workspace you've previously deployed in Azure. Be sure to fill in the settings below which can be retrieved by running `az quantum workspace show`.

In [1]:
from azure.quantum import Workspace

# Copy the settings for your workspace below
workspace = Workspace (
    subscription_id = "",  # Add your subscription_id
    resource_group = "",   # Add your resource_group
    name = "",             # Add your workspace name
    location = ""          # Add your workspace name
)

workspace.login()

Next, we'll define a function that takes an array of container weights and returns a `Problem` object that represents the cost function.

In [2]:
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term

def create_problem_for_container_weights(container_weights: List[int]) -> Problem:
    terms: List[Term] = []

    # Expand the squared summation
    for i in range(len(container_weights)):
        for j in range(len(container_weights)):
            if i == j:
                # Skip the terms where i == j as they form constant terms in an Ising problem and can be disregarded:
                # w_i∗w_j∗x_i∗x_j = w_i​*w_j∗(x_i)^2 = w_i∗w_j​​
                # for x_i = x_j, x_i ∈ {1, -1}
                continue
            
            terms.append(
                Term(
                    c = container_weights[i] * container_weights[j],
                    indices = [i, j]
                )
            )

    # Return an Ising-type problem
    return Problem(name="Ship Sample Problem", problem_type=ProblemType.ising, terms=terms)

Next, we'll define the list of containers and their weights and instantiate a problem:

In [3]:
# This array contains a list of the weights of the containers
container_weights = [1, 5, 9, 21, 35, 5, 3, 5, 10, 11]

# Create a problem for the list of containers:
problem = create_problem_for_container_weights(container_weights)

Now, submit it to Azure using the ParallelTempering solver:

> We'll use Parameter-Free Parallel Tempering with a timeout of 100 seconds. Solver selection and tuning is beyond the scope of this tutorial.

In [4]:
from azure.quantum.optimization import ParallelTempering
import time

# Instantiate a solver to solve the problem. 
solver = ParallelTempering(workspace, timeout=100)

# Optimize the problem
print('Submitting problem...')
start = time.time()
result = solver.optimize(problem)
time_elapsed = time.time() - start
print(f'Result in {time_elapsed} seconds: ', result)

Submitting problem...
.Result in 8.879977226257324 seconds:  {'version': '1.0', 'configuration': {'0': 1, '1': -1, '2': 1, '3': 1, '4': -1, '5': -1, '6': -1, '7': -1, '8': 1, '9': 1}, 'cost': -2052.0, 'parameters': {'all_betas': [0.00020408163265306123, 0.0010031845282727856, 0.004931258069052868, 0.024240112818991428, 0.00020408163265306123, 0.00041416312947479666, 0.0008405023793001501, 0.0017057149691356173, 0.0034615768230851457, 0.007024921700835206, 0.014256371424073268, 0.028931870679351317, 0.058714319100389226, 0.00020408163265306123, 0.0003216601955060876, 0.000506979878727771, 0.0007990687098552142, 0.0012594401274306443, 0.001985047612326009, 0.003128702935041415, 0.0049312580690528685, 0.007772328229454337, 0.012250238227336452, 0.019308028713685834, 0.030432059025318557, 0.04796503207311015, 0.07559936381105262, 0.00020408163265306123, 0.0002853639172320586, 0.0003990195697643234, 0.0005579423586529702, 0.000780161423569038, 0.0010908866075247, 0.0015253684103382742, 0.00

Lastly, print out a summary of what the solution means:

In [5]:
def print_result_summary(result):
    # Print a summary of the result
    ship_a_weight = 0
    ship_b_weight = 0
    for container in result['configuration']:
        container_assignment = result['configuration'][container]
        container_weight = container_weights[int(container)]
        ship = ''
        if container_assignment == 1:
            ship = 'A'
            ship_a_weight += container_weight
        else:
            ship = 'B'
            ship_b_weight += container_weight

        print(f'Container {container} with weight {container_weight} was placed on Ship {ship}')

    print(f'\nTotal weights: \n\tShip A: {ship_a_weight} tonnes \n\tShip B: {ship_b_weight} tonnes\n')

print_result_summary(result)

Container 0 with weight 1 was placed on Ship A
Container 1 with weight 5 was placed on Ship B
Container 2 with weight 9 was placed on Ship A
Container 3 with weight 21 was placed on Ship A
Container 4 with weight 35 was placed on Ship B
Container 5 with weight 5 was placed on Ship B
Container 6 with weight 3 was placed on Ship B
Container 7 with weight 5 was placed on Ship B
Container 8 with weight 10 was placed on Ship A
Container 9 with weight 11 was placed on Ship A

Total weights: 
	Ship A: 52 tonnes 
	Ship B: 53 tonnes



## Improving the Cost Function
The cost function we've built works well so far, but let's take a closer look at the `Problem` that was generated:


In [6]:
print(f'The problem has {len(problem.terms)} terms for {len(container_weights)} containers:')
print(problem.terms)

The problem has 90 terms for 10 containers:
[{'c': 5, 'ids': [0, 1]}, {'c': 9, 'ids': [0, 2]}, {'c': 21, 'ids': [0, 3]}, {'c': 35, 'ids': [0, 4]}, {'c': 5, 'ids': [0, 5]}, {'c': 3, 'ids': [0, 6]}, {'c': 5, 'ids': [0, 7]}, {'c': 10, 'ids': [0, 8]}, {'c': 11, 'ids': [0, 9]}, {'c': 5, 'ids': [1, 0]}, {'c': 45, 'ids': [1, 2]}, {'c': 105, 'ids': [1, 3]}, {'c': 175, 'ids': [1, 4]}, {'c': 25, 'ids': [1, 5]}, {'c': 15, 'ids': [1, 6]}, {'c': 25, 'ids': [1, 7]}, {'c': 50, 'ids': [1, 8]}, {'c': 55, 'ids': [1, 9]}, {'c': 9, 'ids': [2, 0]}, {'c': 45, 'ids': [2, 1]}, {'c': 189, 'ids': [2, 3]}, {'c': 315, 'ids': [2, 4]}, {'c': 45, 'ids': [2, 5]}, {'c': 27, 'ids': [2, 6]}, {'c': 45, 'ids': [2, 7]}, {'c': 90, 'ids': [2, 8]}, {'c': 99, 'ids': [2, 9]}, {'c': 21, 'ids': [3, 0]}, {'c': 105, 'ids': [3, 1]}, {'c': 189, 'ids': [3, 2]}, {'c': 735, 'ids': [3, 4]}, {'c': 105, 'ids': [3, 5]}, {'c': 63, 'ids': [3, 6]}, {'c': 105, 'ids': [3, 7]}, {'c': 210, 'ids': [3, 8]}, {'c': 231, 'ids': [3, 9]}, {'c': 35, 'ids'

That's alot of terms for just 10 containers! On closer inspection however, you'll note that there are essentially duplicated terms that result from having squared the right hand side of the equation. For example, look at the last term: `{'w': 110, 'ids': [9, 8]}`. If you look through the rest of the terms, you'll find a symmetrical copy of this term: `{'w': 110, 'ids': [8, 9]}`.

This duplicate encodes the exact same information in our cost function. However, because we don't actually care about the value of the cost function (just the shape), we can omit these terms too by a slight modification to our cost function:

$$ H^2 = \Large(\sum_{i<j} w_{i} x_{i})^2 $$

In code, this means a small modification to the `create_simplified_problem_for_container_weights` function:

In [7]:
def create_simplified_problem_for_container_weights(container_weights: List[int]) -> Problem:
    terms: List[Term] = []

    # Expand the squared summation
    for i in range(len(container_weights)-1):
        for j in range(i+1, len(container_weights)):
            terms.append(
                Term(
                    c = container_weights[i] * container_weights[j],
                    indices = [i, j]
                )
            )

    # Return an Ising-type problem
    return Problem(name="Ship Sample Problem (Simplified)", problem_type=ProblemType.ising, terms=terms)

Let's check that this creates a smaller problem:

In [8]:
# Create the simplified problem
simplified_problem = create_simplified_problem_for_container_weights(container_weights)
print(f'The simplified problem has {len(simplified_problem.terms)} terms')

The simplified problem has 45 terms


Success! The problem has half as many terms. Now let's run it and verify the result:

In [9]:
# Optimize the problem
print('Submitting simplified problem...')
start = time.time()
simplified_result = solver.optimize(simplified_problem)
time_elapsed_simplified = time.time() - start
print(f'Result in {time_elapsed_simplified} seconds:\n{simplified_result}\n')
print_result_summary(simplified_result)


Submitting simplified problem...
..Result in 9.023997783660889 seconds:
{'version': '1.0', 'configuration': {'0': 1, '1': -1, '2': 1, '3': 1, '4': -1, '5': -1, '6': -1, '7': -1, '8': 1, '9': 1}, 'cost': -1026.0, 'parameters': {'all_betas': [0.00040816326530612246, 0.002006369056545571, 0.009862516138105735, 0.048480225637982856, 0.00040816326530612246, 0.0008283262589495933, 0.0016810047586003002, 0.0034114299382712347, 0.006923153646170291, 0.014049843401670412, 0.028512742848146536, 0.05786374135870263, 0.11742863820077845, 0.00040816326530612246, 0.0006433203910121752, 0.001013959757455542, 0.0015981374197104284, 0.0025188802548612886, 0.003970095224652018, 0.00625740587008283, 0.009862516138105737, 0.015544656458908674, 0.024500476454672904, 0.03861605742737167, 0.060864118050637114, 0.0959300641462203, 0.15119872762210523, 0.00040816326530612246, 0.0005707278344641172, 0.0007980391395286468, 0.0011158847173059405, 0.001560322847138076, 0.0021817732150494, 0.0030507368206765485, 0.

As you can see, the quality of the solution is the same for both cost functions - the ships are loaded within 1 tonne of each other (a perfect solution does not exist). This reveals an important fact about using QIO solvers: it is often possible (and neccesary) to optimize the cost function in order to generate more optimal solutions more quickly.